# Test

In [ ]:
# divide terms by the starting character
df_term_a = df_insurance_terms_2[df_insurance_terms_2['term'].str.startswith('a')]

In [ ]:
df_term_a.head()

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def normalize_string(string):
    string = string.lower()
    tokenized_words = word_tokenize(string)
    stopwords_removed = [word for word in tokenized_words if word not in stop_words]
    alphabet_string = [word for word in stopwords_removed if word.isalpha()]
    return alphabet_string

In [ ]:
df_term_a.loc[:, 'term'] = df_term_a.apply(lambda row: normalize_string(row['term']), axis=1)
df_term_a.loc[:, 'text'] = df_term_a.apply(lambda row: normalize_string(row['text']), axis=1)

df_term_a.head()

In [ ]:
# lower case
# df_term_a['term'] = df_term_a['term'].str.lower()
# df_term_a['text'] = df_term_a['text'].str.lower()
df_term_a.loc[:,'term'] = df_term_a['term'].str.lower()
df_term_a.loc[:,'text'] = df_term_a['text'].str.lower()

In [ ]:
df_term_a.head()

In [ ]:
# Tokenize
from nltk import word_tokenize
# df_term_a['term_tokenized'] = df_term_a.apply(lambda row: word_tokenize(row['term']), axis=1)
# df_term_a['text_tokenized'] = df_term_a.apply(lambda row: word_tokenize(row['text']), axis=1)
df_term_a.loc[:, 'term_tokenized'] = df_term_a['term'].apply(word_tokenize)
df_term_a.loc[:, 'text_tokenized'] = df_term_a['text'].apply(word_tokenize)

In [ ]:
df_term_a.head()

In [ ]:
before_remove_stopwords = df_term_a.loc[0, 'text_tokenized']

In [ ]:
# Remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')

# df_term_a['term_tokenized'] = df_term_a['term_tokenized'].apply(lambda x: [item for item in x if item not in stop])
# df_term_a['text_tokenized'] = df_term_a['text_tokenized'].apply(lambda x: [item for item in x if item not in stop])
df_term_a.loc[:,'term_tokenized'] = df_term_a['term_tokenized'].apply(lambda x: [item for item in x if item not in stop])
df_term_a.loc[:,'text_tokenized'] = df_term_a['text_tokenized'].apply(lambda x: [item for item in x if item not in stop])



In [ ]:
df_term_a.head()

In [ ]:
after_remove_stopwords = df_term_a.loc[0, 'text_tokenized']

In [ ]:
before_remove_stopwords

In [ ]:
after_remove_stopwords

In [ ]:
for item in before_remove_stopwords:
    if item not in after_remove_stopwords:
        print(item)

In [ ]:
# keep only alphabet
# df_term_a['term_tokenized'] = df_term_a['term_tokenized'].apply(lambda x: [item for item in x if item.isalpha()])
# df_term_a['text_tokenized'] = df_term_a['text_tokenized'].apply(lambda x: [item for item in x if item.isalpha()])
df_term_a.loc[:,'term_tokenized'] = df_term_a['term_tokenized'].apply(lambda x: [item for item in x if item.isalpha()])
df_term_a.loc[:,'text_tokenized'] = df_term_a['text_tokenized'].apply(lambda x: [item for item in x if item.isalpha()])

In [ ]:
df_term_a.head()

In [ ]:
X = documents.values
y = labels.values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

vec_trained = vectorizer.fit_transform(X_train)

vec_trained = vec_trained.toarray()

words = vectorizer.get_feature_names()

vec_test = vectorizer.transform(X_test)

vec_test = vec_test.toarray()

In [ ]:
vec_trained.shape

In [ ]:
words

In [ ]:
vec_test = vectorizer.transform(X_test)

Because there 3261 terms corresponding to 3261 labels. First I categorize the label into different clusters. And I want to know how many unique words in documents and labels? So I can decide the size of vocabulary.

In [ ]:
docs = []
for index, text in documents.iteritems():
#     print(text)
    docs.append(text)

In [ ]:
docs

In [ ]:
from nltk.tokenize import word_tokenize

tokenized = [word_tokenize(text.lower()) for text in docs]
tokenized

In [ ]:
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))

docs = [[word for word in words if  word not in stop] for words in tokenized] # words 是一個列 word 是該列的一個單字
docs

In [ ]:
# docs[0]

vocab = [word for words in docs for word in words]
vocab

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
wordnet = WordNetLemmatizer()
vocab = [wordnet.lemmatize(word) for word in vocab]
vocab

In [ ]:
print(len(vocab))

vocab = set(vocab)
print(len(vocab))

There are 8067 unique words in document.

Wrap above into function

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

def get_vocabulary(docs):
    tokenized = [word_tokenize(text.lower()) for text in docs]
    stop = set(stopwords.words('english'))
    docs = [[word for word in words if word not in stop] for words in tokenized]
    vocab = [word for words in docs for word in words]

    wordnet = WordNetLemmatizer()
    vocab = [wordnet.lemmatize(word) for word in vocab]
    vocab = set(vocab)
    return vocab

In [ ]:
terms = []
texts = []

for index, row in df_insurance_terms_2.iterrows():
#     print(index, row[0], row[1])
    terms.append(row[0])
    texts.append(row[1])
    
vocab_terms = get_vocabulary(terms)
vocab_texts = get_vocabulary(texts)

In [ ]:
print(len(vocab_terms), len(vocab_texts))

So there are 2873 unique words in terms and 8067 unique words in texts.
Let me set vocabulary for terms and texts to be (1/3)*size in the analysis.

In [ ]:
max_size_terms = 2873//3
max_size_texts = 8067//3
print(max_size_terms, max_size_texts)

In [ ]:
X = documents.values
y = labels.values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

In [ ]:
y_train

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_X = TfidfVectorizer(stop_words='english', max_features=max_size_texts)
vec_X_trained = vectorizer_X.fit_transform(X_train)

vectorizer_y = TfidfVectorizer(stop_words='english', max_features=max_size_terms)
vec_y_trained = vectorizer_y.fit_transform(y_train)

In [ ]:
vocab_X = vectorizer_X.get_feature_names()
vocab_X

In [ ]:
vocab_y = vectorizer_y.get_feature_names()
vocab_y

Use grid search to find the optimized k for KMeans.

In [ ]:
vec_labels = vectorizer_y.transform(labels)

from sklearn.cluster import KMeans
n=8
kmeans = KMeans(n_clusters=n).fit(vec_trained)
print(kmeans.labels_.size)
kmeans.labels_

In [ ]:
from sklearn.model_selection import GridSearchCV

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

_count_vectorizer_labels = CountVectorizer()
_count_vectorizer_defs = CountVectorizer()

labels=['battery',
        'car, van, suv',
        'imaginary thing']
defs=['device that uses chemicals to store an electric charge',
      'passenger vehicle with 4 wheels and no truck bed',
      'device that stores a vehicle or chemicals']

In [2]:
count_vectors_labels = _count_vectorizer_labels.fit_transform(labels).todense()
count_vectors_defs = _count_vectorizer_defs.fit_transform(defs).todense()

In [3]:
count_vectors_labels

matrix([[1, 0, 0, 0, 0, 0],
        [0, 1, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 0]], dtype=int64)

In [4]:
count_vectors_defs

matrix([[1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]],
       dtype=int64)

In [7]:
count_vectors_labels.shape

(3, 6)

In [11]:
count_vectors_defs.shape

(3, 19)

In [5]:
row = 2
labels_ct = ([count_vectors_labels[row,n] for n in range(count_vectors_labels.shape[1])])

labels_ct

[0, 0, 1, 0, 1, 0]

In [9]:
labels_feature = (_count_vectorizer_labels.get_feature_names())

labels_feature

['battery', 'car', 'imaginary', 'suv', 'thing', 'van']

In [10]:
z = zip(labels_ct, labels_feature)
for ct,feat in z:
   print(f'{ct}\t{feat}')

# 0 battery
# 0 car
# 1 imaginary
# 0 suv
# 1 thing
# 0 van


0	battery
0	car
1	imaginary
0	suv
1	thing
0	van


In [12]:
row = 2
defs_ct = ([count_vectors_defs[row,n] for n in range(count_vectors_defs.shape[1])])

defs_ct

[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0]

In [13]:
defs_feature = (_count_vectorizer_defs.get_feature_names())

defs_feature

['an',
 'and',
 'bed',
 'charge',
 'chemicals',
 'device',
 'electric',
 'no',
 'or',
 'passenger',
 'store',
 'stores',
 'that',
 'to',
 'truck',
 'uses',
 'vehicle',
 'wheels',
 'with']

In [14]:
z = zip(defs_ct, defs_feature)
for ct,feat in z:
   print(f'{ct}\t{feat}')

# 0 an
# 0 and
# 0 bed
# 0 charge
# 1 chemicals
# 1 device
# 0 electric
# 0 no
# 1 or
# 0 passenger
# 0 store
# 1 stores
# 1 that
# 0 to
# 0 truck
# 0 uses
# 1 vehicle
# 0 wheels
# 0 with



0	an
0	and
0	bed
0	charge
1	chemicals
1	device
0	electric
0	no
1	or
0	passenger
0	store
1	stores
1	that
0	to
0	truck
0	uses
1	vehicle
0	wheels
0	with


In [15]:
from sklearn.neural_network import MLPRegressor
defs_labels_regressor = MLPRegressor()
defs_labels_regressor.fit(count_vectors_defs, count_vectors_labels)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [16]:
def words_in_label(inpt):
   inpt_len = len(inpt.split(' '))
   pred = defs_labels_regressor.predict(_count_vectorizer_defs.transform([inpt]))
   pred = [pred[0,n] for n in range(pred.shape[1])]
   z = zip(labels_feature, pred)
   for feat,prob in z:
       print(f'{feat:20}{prob:0.4f}')

In [17]:
inpt = 'device that uses chemicals to store an exciting charge'
words_in_label(inpt)

# battery             0.6707
# car                 0.0414
# imaginary           0.0636
# suv                 0.0169
# thing               0.0724
# van                 -0.0054



battery             0.7931
car                 0.0081
imaginary           -0.0939
suv                 -0.0501
thing               0.1312
van                 0.0078


In [18]:
inpt = 'passenger vehicle with 4 feet and no truck bed'
words_in_label(inpt)

# battery             -0.0071
# car                 1.0766
# imaginary           0.0989
# suv                 0.8496
# thing               0.1142
# van                 1.0061



battery             -0.0194
car                 0.8085
imaginary           0.0579
suv                 0.8058
thing               0.0993
van                 0.6642


In [19]:

inpt = 'device that uses chemicals and no bed'
words_in_label(inpt)

# battery             0.5551
# car                 0.2812
# imaginary           0.4854
# suv                 0.0997
# thing               0.7767
# van                 0.8325

battery             0.4998
car                 0.1810
imaginary           -0.1432
suv                 0.4232
thing               0.5352
van                 0.6950


In [ ]:
# Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Try regression model

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1.0, penalty='l2')

clf.fit(X_train, y_train)

In [ ]:
The dimension of X_train is (2282, 7400) and y_train is (2282, 2716)

But in order to apply logistic regression, the y_train should be (2282,)

I need to split the tokens in a row into different rows with same text.

|norm term              | norm text                                           |
|:---------------------:|:---------------------------------------------------:|
|automatic premium loan	| an optional provision in life insurance that a...   |

should become 


|norm term              | norm text                                           |
|:---------------------:|:---------------------------------------------------:|
|automatic premium loan	| an optional provision in life insurance that a...   |
|premium                | an optional provision in life insurance that a...   |
|loan                	| an optional provision in life insurance that a...   |

In [ ]:
# Save the normalized term and text to csv

In [ ]:
df_cnt_norm = df_cnt[['norm_term', 'norm_text']]
df_cnt_norm.head()

In [ ]:
df_cnt_norm.to_csv('norm.csv', index=False)